# import libs

In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import tensorflow as tf
import optuna
import dataset
import numpy as np
import keras
import joblib
from datetime import datetime
from keras import layers
import keras_cv

Using TensorFlow backend


/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##  define constants and create dataset


In [2]:
NTRIALS = 300
BATCH_SIZE = 64
EPOCHS = 100

train_images_rgx = "../../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"

layer_options = ["Conv2D", "Depthwise2D", "Pointwise2D"]
activation_options = ["linear", "relu", "tanh"]
filter_size_options = [3, 4, 5, 8, 16]

train_dataset, val_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE
)

## define create model function


In [3]:
def clip_filter_size(fs, ih):
    return min(fs, ih)


def create_layer(trial, layer_type, layer_id, max_stride, input_h_size):
    if layer_type == "Conv2D":
        num_filters = trial.suggest_int(
            f"layer_{layer_id}_conv2d_filters", 4, 32, step=4
        )
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_conv2d_fsize", filter_size_options
        )

        stride_size = trial.suggest_int(f"layer_{layer_id}_conv2d_ssize", 1, max_stride)
        layer = layers.Conv2D(
            num_filters, clip_filter_size(filter_size, input_h_size), stride_size
        )

    elif layer_type == "Depthwise2D":
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_dwconv2d_fsize", filter_size_options
        )
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_dwconv2d_ssize", 1, max_stride
        )
        layer = layers.DepthwiseConv2D(
            clip_filter_size(filter_size, input_h_size), stride_size
        )

    elif layer_type == "Pointwise2D":
        num_filters = trial.suggest_int(
            f"layer_{layer_id}_pwconv2d_filters", 4, 32, step=4
        )
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_pwconv2d_ssize", 1, max_stride
        )
        layer = layers.Conv2D(num_filters, 1, stride_size)

    return layer


def create_augmentation_layers(trial, x):
    # augmix_severity = trial.suggest_float("augmix_severity", 0.0, 0.5)
    brightness_factor = trial.suggest_float("brightness_factor", 0, 0.2)
    gaussian_noise_factor = trial.suggest_float("gaussian_noise_factor", 0, 0.2)
    translate_factor = trial.suggest_float("translate_factor", 0, 0.2)

    # x = keras_cv.layers.AugMix(value_range=(0, 1), severity=augmix_severity)(x)
    x = layers.RandomBrightness(brightness_factor, (0, 1.0))(x)
    x = layers.RandomFlip()(x)
    x = layers.RandomTranslation(translate_factor, translate_factor)(x)
    x = layers.GaussianNoise(gaussian_noise_factor)(x)

    return x


def create_model(trial):
    # add the input layer
    x = x_in = layers.Input(shape=(256, 256, 3))

    x = create_augmentation_layers(trial, x)

    # suggent a number of layer
    num_layers = trial.suggest_categorical("num_layers", [1, 2, 4])

    # limits the stride
    max_stride = 16 // num_layers

    # convolutional layers
    for i in range(num_layers):
        layer_selected = trial.suggest_categorical(f"layer_{i}", layer_options)
        layer = create_layer(
            trial, layer_selected, i, max_stride, input_h_size=x.shape[1]
        )
        x = layer(x)
        act_selected = trial.suggest_categorical(f"activation_{i}", activation_options)
        x = layers.Activation(act_selected)(x)

    # output layer
    x = layers.Flatten()(x)
    x = layers.Dropout(trial.suggest_float("output_dropout", 0.3, 0.6))(x)
    x = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(x_in, x)

# if testing


```
class MockTrial:
    def suggest_int(self, name, min, max, step=None, log=False):
        return np.random.randint(min, max)

    def suggest_float(self, name, min, max, step=None, log=False):
        return np.random.uniform(min, max)

    def suggest_categorical(self, name, options):
        print(name)
        return options[np.random.randint(0, len(options) - 1)]

    def suggest_discrete_uniform(self, name, min, max, q):
        return (np.random.randint(min, max) // q) * q

model = create_model(MockTrial())
model.build(input_shape=(BATCH_SIZE, 256, 256, 3))
model.summary()
model.save("saved_models/test/")
```

In [4]:
def create_optimizer(trial):
    lr = trial.suggest_float("lr_inicial", 1e-4, 2e-3, log=True)
    optimizer = keras.optimizers.Adam(lr)
    return optimizer


def create_model_checkpointer(save_path):
    return keras.callbacks.ModelCheckpoint(
        save_path + "weights/",
        "val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )


def crate_early_stopper():
    return keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0.1,
        patience=20,
        verbose=0,
        mode="max",
        start_from_epoch=50,
    )



def objective(trial):
    BATCH_SIZE = trial.suggest_int("batchsize", 32, 128, step=32)
    
    # load data
    train_dataset, val_dataset, num_classes = dataset.create_datasets(
        train_images_rgx, valid_images_rgx, BATCH_SIZE
    )

    # Build model and optimizer.
    model = create_model(trial)
    assert isinstance(model, keras.Model), "model is not a model"

    optimizer = create_optimizer(trial)

    # compile the model
    model.compile(
        optimizer, loss=keras.losses.categorical_crossentropy, metrics=["accuracy"]
    )

    # crate model checkpointer
    saved_best_model_path = f"saved_models/{EXP_ID}/best_model_t{trial._trial_id}/"
    model.save(saved_best_model_path, True, "tf")  # save all model

    model_chkp = create_model_checkpointer(saved_best_model_path)
    early_stopper = crate_early_stopper()

    # Training and validating cycle.
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=[model_chkp, early_stopper],
        verbose="auto",
    )
    
    # load best model
    model.load_weights(saved_best_model_path + "weights/")    

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(val_dataset, verbose=0)
    
    # Return best model validation accuracy.
    return score[1]


def search():
    study = optuna.create_study(
        direction="maximize",
        sampler=optuna.samplers.TPESampler(),
    )
    study.optimize(objective, n_trials=NTRIALS)
    return study

# start search

In [5]:
EXP_ID = "exp4_with_aug"
keras.utils.disable_interactive_logging()

# start searching for a model
study = search()

# save the study
now = datetime.now()
joblib.dump(study, "saved_studies/" + now.strftime("%Y:%m:%d:%H:%M:%S") + ".pkl")

[I 2024-01-30 21:11:15,213] A new study created in memory with name: no-name-c9c47ad8-82dc-4da8-81af-197baebf7937


INFO:tensorflow:Assets written to: saved_models/exp4_with_aug/best_model_t0/assets


INFO:tensorflow:Assets written to: saved_models/exp4_with_aug/best_model_t0/assets
I0000 00:00:1706659878.687900 3088249 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
[I 2024-01-30 21:11:32,964] Trial 0 finished with value: 0.699999988079071 and parameters: {'batchsize': 128, 'brightness_factor': 0.014321297262167399, 'gaussian_noise_factor': 0.11424661493919946, 'translate_factor': 0.053700753742687235, 'num_layers': 2, 'layer_0': 'Conv2D', 'layer_0_conv2d_filters': 16, 'layer_0_conv2d_fsize': 8, 'layer_0_conv2d_ssize': 7, 'activation_0': 'linear', 'layer_1': 'Conv2D', 'layer_1_conv2d_filters': 12, 'layer_1_conv2d_fsize': 4, 'layer_1_conv2d_ssize': 2, 'activation_1': 'linear', 'output_dropout': 0.3086555258231815, 'lr_inicial': 0.0008812211746603344}. Best is trial 0 with value: 0.699999988079071.


INFO:tensorflow:Assets written to: saved_models/exp4_with_aug/best_model_t1/assets


INFO:tensorflow:Assets written to: saved_models/exp4_with_aug/best_model_t1/assets
[I 2024-01-30 21:11:46,295] Trial 1 finished with value: 0.6000000238418579 and parameters: {'batchsize': 96, 'brightness_factor': 0.06562384780797118, 'gaussian_noise_factor': 0.052522566217725886, 'translate_factor': 0.030106894776780947, 'num_layers': 2, 'layer_0': 'Pointwise2D', 'layer_0_pwconv2d_filters': 12, 'layer_0_pwconv2d_ssize': 5, 'activation_0': 'linear', 'layer_1': 'Pointwise2D', 'layer_1_pwconv2d_filters': 8, 'layer_1_pwconv2d_ssize': 5, 'activation_1': 'relu', 'output_dropout': 0.4164676880457159, 'lr_inicial': 0.0002584176626691037}. Best is trial 0 with value: 0.699999988079071.


...

INFO:tensorflow:Assets written to: saved_models/exp4_with_aug/best_model_t298/assets


INFO:tensorflow:Assets written to: saved_models/exp4_with_aug/best_model_t298/assets
[I 2024-01-31 00:27:13,592] Trial 298 finished with value: 0.8199999928474426 and parameters: {'batchsize': 32, 'brightness_factor': 0.05945385647687181, 'gaussian_noise_factor': 0.01242114951052122, 'translate_factor': 0.022156819639210474, 'num_layers': 1, 'layer_0': 'Conv2D', 'layer_0_conv2d_filters': 8, 'layer_0_conv2d_fsize': 4, 'layer_0_conv2d_ssize': 3, 'activation_0': 'linear', 'output_dropout': 0.4447693382258449, 'lr_inicial': 0.0009814101544450638}. Best is trial 103 with value: 1.0.


INFO:tensorflow:Assets written to: saved_models/exp4_with_aug/best_model_t299/assets


INFO:tensorflow:Assets written to: saved_models/exp4_with_aug/best_model_t299/assets
[I 2024-01-31 00:27:42,374] Trial 299 finished with value: 0.9800000190734863 and parameters: {'batchsize': 32, 'brightness_factor': 0.05497898531177192, 'gaussian_noise_factor': 0.023892401548890496, 'translate_factor': 0.015405942561736413, 'num_layers': 4, 'layer_0': 'Conv2D', 'layer_0_conv2d_filters': 8, 'layer_0_conv2d_fsize': 4, 'layer_0_conv2d_ssize': 2, 'activation_0': 'linear', 'layer_1': 'Conv2D', 'layer_1_conv2d_filters': 16, 'layer_1_conv2d_fsize': 4, 'layer_1_conv2d_ssize': 1, 'activation_1': 'relu', 'layer_2': 'Conv2D', 'layer_2_conv2d_filters': 20, 'layer_2_conv2d_fsize': 3, 'layer_2_conv2d_ssize': 1, 'activation_2': 'relu', 'layer_3': 'Depthwise2D', 'layer_3_dwconv2d_fsize': 5, 'layer_3_dwconv2d_ssize': 4, 'activation_3': 'linear', 'output_dropout': 0.5191597951901296, 'lr_inicial': 0.0013615855713598232}. Best is trial 103 with value: 1.0.


['saved_studies/2024:01:31:00:27:42.pkl']

# visualizations

In [6]:
print("Number of finished trials: ", len(study.trials))

trial = study.best_trial
print(f"Best trial id: {trial._trial_id}")

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  300
Best trial id: 103
  Value:  1.0
  Params: 
    batchsize: 32
    brightness_factor: 0.07810315809868569
    gaussian_noise_factor: 0.012366729993256585
    translate_factor: 0.03243066408331395
    num_layers: 2
    layer_0: Conv2D
    layer_0_conv2d_filters: 8
    layer_0_conv2d_fsize: 4
    layer_0_conv2d_ssize: 3
    activation_0: linear
    layer_1: Conv2D
    layer_1_conv2d_filters: 8
    layer_1_conv2d_fsize: 5
    layer_1_conv2d_ssize: 1
    activation_1: relu
    output_dropout: 0.4566767363346691
    lr_inicial: 0.0011698580163583688


In [7]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [8]:
plot_optimization_history(study)

In [20]:
plot_parallel_coordinate(
    study,
    [
        "lr_inicial",
        "num_layers",
        "layer_0",
        "activation_0",
    ],
)

In [12]:
plot_param_importances(study)

In [ ]:
import keras

model = keras.models.load_model("/home/webphy/Desktop/dnn_processor/neural_network/automl/saved_models/exp2_with_aug/best_model_t185")

for l in model.layers:
    print(l.get_config())